In [4]:
import math

def div_round_up(a, b):
    return (a + b - 1) // b

def grid_scale(level, log2_per_level_scale, base_resolution):
    return base_resolution * (2 ** (level * log2_per_level_scale))

def grid_resolution(scale):
    return int(math.ceil(scale))

def next_multiple(value, multiple):
    return ((value + multiple - 1) // multiple) * multiple

def powi(base, exponent):
    return int(base ** exponent)

def grid_encoding(config):
    L = config["n_levels"]
    dim = config["n_features_per_level"]
    log2_hashmap_size = config["log2_hashmap_size"]
    base_resolution = config["base_resolution"]
    per_level_scale = config["per_level_scale"]
    
    N_FEATURES_PER_LEVEL = dim
    MAX_N_LEVELS = 32
    N_POS_DIMS = 3  # Assuming 3D position dimensions
    
    m_n_levels = div_round_up(L, N_FEATURES_PER_LEVEL)
    offset = 0
    
    if m_n_levels > MAX_N_LEVELS:
        raise RuntimeError(f"GridEncoding: m_n_levels={m_n_levels} must be at most MAX_N_LEVELS={MAX_N_LEVELS}")
    
    m_offset_table = [0] * m_n_levels
    
    for i in range(m_n_levels):
        # Compute number of dense params required for the given level
        resolution = grid_resolution(grid_scale(i, math.log2(per_level_scale), base_resolution))
        
        max_params = (2 ** 31) - 1  # Equivalent to std::numeric_limits<uint32_t>::max() / 2
        params_in_level = min(pow(resolution, N_POS_DIMS), max_params)
        
        # Make sure memory accesses will be aligned
        params_in_level = next_multiple(params_in_level, 8)
        
        grid_type = config.get("otype", "Hash")
        
        if grid_type == "Dense":
            pass  # No-op
        elif grid_type == "Tiled":
            # If tiled grid needs fewer params than dense, then use fewer and tile.
            params_in_level = min(params_in_level, powi(base_resolution, N_POS_DIMS))
        elif grid_type == "Hash":
            # If hash table needs fewer params than dense, then use fewer and rely on the hash.
            params_in_level = min(params_in_level, (1 << log2_hashmap_size))
        else:
            raise RuntimeError(f"GridEncoding: invalid grid type {grid_type}")
        
        m_offset_table[i] = offset
        offset += params_in_level
        
        print(f"GridEncoding at level {i}: resolution={resolution} params_in_level={params_in_level}")
    
    return m_offset_table

# Example usage
config = {
    "otype": "HashGrid",
    "n_levels": 16,
    "n_features_per_level": 2,
    "log2_hashmap_size": 14,
    "base_resolution": 16,
    "per_level_scale": 2
}

m_offset_table = grid_encoding(config)
print("Offset Table:", m_offset_table)

RuntimeError: GridEncoding: invalid grid type HashGrid

In [7]:
import math

def next_multiple(value, multiple):
    """Round `value` up to the nearest multiple of `multiple`."""
    return ((value + multiple - 1) // multiple) * multiple

def grid_scale(level, per_level_scale, base_resolution):
    """
    Python equivalent of:
       grid_scale(i, std::log2(per_level_scale), base_resolution)
    If per_level_scale=2, this effectively becomes base_resolution * 2^level.
    """
    return int(base_resolution * (per_level_scale ** level))

def compute_grid_offsets(cfg, N_POS_DIMS=3):
    """
    Translates the C++ snippet's logic into Python, returning:
      - offset_table: list of offsets per level
      - total_params: sum of all params_in_level

    cfg is a dictionary containing:
      - otype: "HashGrid" / "DenseGrid" / "TiledGrid" etc.
      - n_levels
      - n_features_per_level
      - log2_hashmap_size
      - base_resolution
      - per_level_scale
    """

    # Unpack configuration
    otype               = cfg["otype"]  # e.g. "HashGrid"
    n_levels            = cfg["n_levels"]
    n_features_per_level = cfg["n_features_per_level"]
    log2_hashmap_size   = cfg["log2_hashmap_size"]
    base_resolution     = cfg["base_resolution"]
    per_level_scale     = cfg["per_level_scale"]

    # (Optional checks, similar to C++ throws)
    # e.g., check if n_levels <= some MAX_N_LEVELS
    # if n_levels > 16:
    #     raise ValueError(f"n_levels={n_levels} exceeds maximum allowed")

    offset_table = []
    offset = 0

    # Simulate the "max_params" check for 32-bit safety
    # C++ used std::numeric_limits<uint32_t>::max() / 2
    max_params_32 = (1 << 31) - 1

    for level in range(n_levels):
        # 1) Compute resolution for this level
        resolution = grid_scale(level, per_level_scale, base_resolution)

        # 2) params_in_level = resolution^N_POS_DIMS (capped by max_params_32)
        raw_params = resolution ** N_POS_DIMS
        params_in_level = raw_params if raw_params <= max_params_32 else max_params_32

        # 3) Align to multiple of 8
        params_in_level = next_multiple(params_in_level, 8)* n_features_per_level

        # 4) Adjust based on grid type
        if otype == "DenseGrid":
            # No-op
            pass
        elif otype == "TiledGrid":
            # Tiled can’t exceed base_resolution^N_POS_DIMS
            tiled_max = (base_resolution ** N_POS_DIMS)
            params_in_level = min(params_in_level, tiled_max)
        elif otype == "HashGrid":
            # Hash grid can't exceed 2^log2_hashmap_size
            hashed_max = (1 << log2_hashmap_size)
            params_in_level = min(params_in_level, hashed_max)
        else:
            raise RuntimeError(f"Invalid grid type '{otype}'")

        # 5) Store offset for this level and increment
        offset_table.append(offset)
        offset += params_in_level

        # (Optional debug print)
        print(f"Level={level}, resolution={resolution}, params_in_level={params_in_level}, offset={offset}")

    # offset now points past the last level’s parameters
    return offset_table, offset


# Example usage:
config = {
    "otype": "HashGrid",
    "n_levels": 16,
    "n_features_per_level": 2,
    "log2_hashmap_size": 14,
    "base_resolution": 16,
    "per_level_scale": 2,
}

offsets, total = compute_grid_offsets(config, N_POS_DIMS=3)
print("Offsets:", offsets)
print("Total parameters:", total)


Level=0, resolution=16, params_in_level=8192, offset=8192
Level=1, resolution=32, params_in_level=16384, offset=24576
Level=2, resolution=64, params_in_level=16384, offset=40960
Level=3, resolution=128, params_in_level=16384, offset=57344
Level=4, resolution=256, params_in_level=16384, offset=73728
Level=5, resolution=512, params_in_level=16384, offset=90112
Level=6, resolution=1024, params_in_level=16384, offset=106496
Level=7, resolution=2048, params_in_level=16384, offset=122880
Level=8, resolution=4096, params_in_level=16384, offset=139264
Level=9, resolution=8192, params_in_level=16384, offset=155648
Level=10, resolution=16384, params_in_level=16384, offset=172032
Level=11, resolution=32768, params_in_level=16384, offset=188416
Level=12, resolution=65536, params_in_level=16384, offset=204800
Level=13, resolution=131072, params_in_level=16384, offset=221184
Level=14, resolution=262144, params_in_level=16384, offset=237568
Level=15, resolution=524288, params_in_level=16384, offset=2

In [8]:
import tinycudann as tcnn
encoding = tcnn.Encoding(3, config)
list(encoding.parameters())[0].shape

torch.Size([499712])